# Pretrained  GPT2  Optimum Model Deployment Example

In this notebook, we will run an example of text generation using GPT2 model exported from HuggingFace and deployed with Seldon's MLServer pre-packed server using Optimim library as a runtime.

After we have the module deployed to Kubernetes, we will run a simple load test to evaluate the module inference performance.

In [38]:
!pip install requests

import requests

In [39]:
%%writefile ./model-settings.json
{
    "name": "transformer",
    "implementation": "mlserver_huggingface.HuggingFaceRuntime",
    "parallel_workers": 0,
    "parameters": {
        "extra": {
            "task": "text-generation",
            "pretrained_model": "distilgpt2"
             
        }
    }
}

Overwriting ./model-settings.json


In [40]:
from subprocess import Popen, PIPE, STDOUT
process = Popen(
     ["mlserver","start", "."],
    stdout=PIPE,
    stderr=STDOUT,
    close_fds=True,
)
for line in iter(process.stdout.readline, b''):
    linestr = line.rstrip().decode('utf-8')
    print(linestr)
    if linestr.find("Loaded model")!=-1:
       break
process.stdout.close()


2022-05-17 00:55:44,032 [mlserver] INFO - Using asyncio event-loop policy: uvloop
2022-05-17 00:55:44.904153: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-17 00:55:44.904379: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd
ImportError: numpy.core.multiarray failed to import

The above exception was the direct cause of the following exception:

SystemError: <built-in function __import__> returned a result with an error set
2022-05-17 00:55:46.518842: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-17 00:55:46.519081: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not lo

In [41]:
# Copy model file
inference_request = {
    "inputs": [
        {
          "name": "args",
          "shape": [1],
          "datatype": "BYTES",
          "data": ["I love Artificial Intelligence"],
        }
    ]
}

import time
start_time = time.monotonic()

response = requests.post("http://localhost:8080/v2/models/transformer/infer", json=inference_request).json() 
print(response)
print(f"Elapsed time: {time.monotonic() - start_time}")

{'model_name': 'transformer', 'model_version': None, 'id': '0e7db5bb-dc5e-4478-aa8a-9813f7df6dc4', 'parameters': None, 'outputs': [{'name': 'output', 'shape': [1], 'datatype': 'BYTES', 'parameters': None, 'data': ['[{"generated_text": "I love Artificial Intelligence. But we have yet to see one of these Artificial Intelligence products, and we think we\'re probably never going to see it as a product we\'re considering.\\n\\n\\n\\"It\'s not even been announced yet,\\" a team"}]']}]}
Elapsed time: 0.8587948680506088


In [42]:
process.kill()

#Sentiment Analysis

In [54]:
%%writefile ./model-settings.json
{
    "name": "transformer",
    "implementation": "mlserver_huggingface.HuggingFaceRuntime",
    "parallel_workers": 0,
    "parameters": {
        "extra": {
            "task": "text-classification"
          
           
        }
    }
}

Overwriting ./model-settings.json


In [55]:
from subprocess import Popen, PIPE, STDOUT
process2 = Popen(
     ["mlserver","start", "."],
    stdout=PIPE,
    stderr=STDOUT,
    close_fds=True,
)
for line in iter(process2.stdout.readline, b''):
    linestr = line.rstrip().decode('utf-8')
    print(linestr)
    if linestr.find("Loaded model")!=-1:
       break
process2.stdout.close()


2022-05-17 01:00:23,798 [mlserver] INFO - Using asyncio event-loop policy: uvloop
Traceback (most recent call last):
  File "/home/lenisha/.local/bin/mlserver", line 8, in <module>
    sys.exit(main())
  File "/home/lenisha/.local/lib/python3.8/site-packages/mlserver/cli/main.py", line 78, in main
    root()
  File "/usr/lib/python3/dist-packages/click/core.py", line 764, in __call__
    return self.main(*args, **kwargs)
  File "/usr/lib/python3/dist-packages/click/core.py", line 717, in main
    rv = self.invoke(ctx)
  File "/usr/lib/python3/dist-packages/click/core.py", line 1137, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/lib/python3/dist-packages/click/core.py", line 956, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/lib/python3/dist-packages/click/core.py", line 555, in invoke
    return callback(*args, **kwargs)
  File "/home/lenisha/.local/lib/python3.8/site-packages/mlserver/cli/main.py", line 20, in wrapper
  

In [52]:
# Copy model file
inference_request = {
    "inputs": [
        {
          "name": "args",
          "shape": [1],
          "datatype": "BYTES",
          "data": ["This is awesome!"],
        }
    ]
}

import time
start_time = time.monotonic()

response = requests.post("http://localhost:8080/v2/models/transformer/infer", json=inference_request).json() 
print(response)
print(f"Elapsed time: {time.monotonic() - start_time}")

{'model_name': 'transformer', 'model_version': None, 'id': 'a1978495-5be2-4411-a746-0e7615532c4a', 'parameters': None, 'outputs': [{'name': 'output', 'shape': [1], 'datatype': 'BYTES', 'parameters': None, 'data': ['[{"label": "LABEL_0", "score": 0.9389998912811279}]']}]}
Elapsed time: 0.03840390901314095


In [53]:
process2.kill()